In [ ]:
import numpy as np
import os
import pandas as pd
import PIL
import PIL.Image
from sklearn.utils import shuffle
from matplotlib import pyplot as plt

from keras.models import Sequential, Model
from keras.layers import Input, merge, Activation
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.callbacks import CSVLogger, ModelCheckpoint

import base64
from datetime import datetime
import os
import shutil
import numpy as np
import socketio
import eventlet.wsgi
from PIL import Image
from flask import Flask
from io import BytesIO

# Intro

We will comunicate with the simulator using a simple Flask Web App using HTTP requests. 

We will first load the model weights we have trained earlier, then create a simple Flask Web app to comunicate with the simulator. What will happen is that the simulator will keep sending us images of what the car "sees" and we'll send it back our model prediction, which was the steering direction

### Side Note 

We need to apply the same transformation here as we did on the training data. If you remember, we defined a ***crop()*** function in order to keep the just the road from the images. As a result, we must to the same thing here, when making inferences on new data.

We have copy-pasted below the function used in the previous notebook, and if by any chance you used another values for cropping, please replace them here as well

In [ ]:
def crop(image):
    return image[60:-25, :, :]

# Load model 

### Information

We need to read the model weights and the model's architecture from the files we have written previously. We have provided a simple helper function called ***load_model_from_json_and_weights*** which will to exactly that

### Provided code

In [ ]:
def load_model_from_json_and_weights(json_path, weights_path):
    with open(json_path, 'r') as json_file:
        loaded_model_json = json_file.read()

    model = model_from_json(loaded_model_json)
    model.load_weights(weights_path)

    return model


### Code Example

You can use the given function as simple as this:
```python
model = load_model_from_json_and_weights(JSON_PATH, WEIGHTS_PATH)
```

In [ ]:
#Your code here


[Answer](http://localhost:8887/edit/clujai_workspace/clujAI-self-driving-car-workshop/answers/load_model_answer.py)

In [ ]:
model = load_model_from_json_and_weights(JSON_PATH, WEIGHTS_PATH)

# Start Web Server

### Information

Just run the code below in order to initialize the server

### Provided Code

In [ ]:
def start_web_server()
    sio = socketio.Server()
    app = Flask(__name__)


    @sio.on('telemetry')
    def telemetry(sid, data):
        max_speed = 25
        if data:
            # The current steering angle of the car
            steering_angle = float(data["steering_angle"])
            # The current throttle of the car, how hard to push peddle
            throttle = float(data["throttle"])
            # The current speed of the car
            speed = float(data["speed"])
            # The current image from the center camera of the car
            image = Image.open(BytesIO(base64.b64decode(data["image"])))
            try:
                image = np.asarray(image)       # from PIL image to numpy array
                image = crop(image) # apply the preprocessing
                image = np.array([image])       # the model expects 4D array
                image = np.transpose(image,(0,3,1,2))

                # predict the steering angle for the image
                steering_angle = float(model.predict(image, batch_size=1))

                throttle = 1.0 - steering_angle**2 - (speed/max_speed)**2

                print('{} {} {}'.format(steering_angle, throttle, speed))
                send_control(steering_angle, throttle)
            except Exception as e:
                print(e)
        else:

            sio.emit('manual', data={}, skip_sid=True)


    @sio.on('connect')
    def connect(sid, environ):
        print("connect ", sid)
        send_control(0, 0)


    def send_control(steering_angle, throttle):
        sio.emit(
            "steer",
            data={
                'steering_angle': steering_angle.__str__(),
                'throttle': throttle.__str__()
            },
            skip_sid=True)


    app = socketio.Middleware(sio, app)
    eventlet.wsgi.server(eventlet.listen(('', 4567)), app)    

In [ ]:
start_web_server()